In [1]:
from funcs.notebook import *

import numpy as np
import pandas as pd
import pytest


orbper =  8.463 # exoplanet.eu
# critical KS statistic D for 99% confidence
g001 = lambda x: 1.62762 / np.sqrt(x)
# critical KS statistic D for 99.9% confidence
g0001 = lambda x: 1.94947 / np.sqrt(x)

In [2]:
# Get K-S test results
res = pd.read_csv("../results/kstests.csv")

# Select only those where we study orbital 
# dependence on the total sample of flares
res = res[(res.period=="Orbit") &
          (res.subsample=="total")] 

# Get total observing time for each Sector out of the table
d = res[["sector","totobs_days"]].iloc[1:]
d = d.set_index("sector").totobs_days.to_dict()

# Get total number of flares for each Sector out of the table
n = res[["sector","nflares"]].iloc[1:]
n = n.set_index("sector").nflares.to_dict()

# Convert to baseline flare frequencies
f1 = n["Sector 1"] / d["Sector 1"]
f2 = n["Sector 27"] / d["Sector 27"]

In [3]:
# How many flares do we need to observe
# to get a significant result?
# More than 0 but probably less than 30000?
x = np.arange(0,30000)


In [4]:
resul = {}

# Cycle through all three analyses (Sectors 1, 27 and 1+27)
for i, row in res.iterrows():
    
    # Get the number of flares we need 
    # to observe in total for a p=0.01 and p=0.001
    N001 = np.argmin(np.abs(g001(x) - row.D))
    N0001 = np.argmin(np.abs(g0001(x) - row.D))
    
    # How many flares are left after subtracting what have already seen?
    dN001, dN0001 = N001 - row.nflares, N0001 - row.nflares
    
    # Calculate for how low we need to observe AU Mic
    # at different cadences to get a significant detection
    resul[row.sector] = {"99%" : {"2 min" : dN001 / f1, 
                                  "20 s" : dN001 / f2}, 
                         "99.9%" : {"2 min" : dN0001 / f1, 
                                    "20 s" : dN0001 / f2}}

In [5]:
# Transform the dict to a table 
# to create a latex output for the paper
reform = {(outerKey, innerKey): values for outerKey, 
          innerDict in resul.items() for innerKey, 
          values in innerDict.items()}

prediction = pd.DataFrame(reform)

In [6]:

stri = prediction.round(1).to_latex(index=True, escape=False, column_format=f"l|ccccc")
stri = stri.replace("\\toprule","\hline")
stri = stri.replace("\\midrule","\hline")
stri = stri.replace("{}","\hline")
stri = stri.replace("\\bottomrule","\hline\n" )
stri = stri.replace("%","\%" )
print(stri)

\begin{tabular}{l|cccccc}
\hline
\hline & \multicolumn{2}{l}{Both Sectors} & \multicolumn{2}{l}{Sector 1} & \multicolumn{2}{l}{Sector 27} \\
\hline &          99\% &  99.9\% &      99\% & 99.9\% &       99\% &  99.9\% \\
\hline
2 min &         62.7 &  117.3 &     12.7 &  29.2 &      63.7 &  108.2 \\
20 s  &         38.3 &   71.6 &      7.8 &  17.8 &      38.9 &   66.1 \\
\hline

\end{tabular}



In [9]:
with open("/home/ekaterina/Documents/002_writing/aumic-flaring-spi-draft/tables/prediction.tex", "w") as f:
    f.write(stri)

In [30]:
48.4490740740741 / orbper

5.724810832337718

In [7]:
117.3 / orbper

13.860333215171925

In [8]:
62.7 /orbper

7.408720311946119